In [1]:

import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import COBYLA


from imblearn.over_sampling import SMOTE

In [ ]:



caminho_do_arquivo = '../data/student-por.csv'
df_completo = pd.read_csv(caminho_do_arquivo, sep=',')

y_grades = df_completo[['G1', 'G2', 'G3']]
X_features = df_completo.drop(columns=['G1', 'G2', 'G3'])


y_final = np.where(y_grades['G3'] >= 10, 1, 0)


X_final = pd.get_dummies(X_features, drop_first=True)


print("Formato features (X_final):", X_final.shape)
print("Distribuição das classes (Aprovado/Reprovado):")
print(pd.Series(y_final).value_counts())

Formato features (X_final): (649, 39)
Distribuição das classes (Aprovado/Reprovado):
1    549
0    100
Name: count, dtype: int64


In [3]:

X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
)


top_features_sp = ['failures', 'higher_yes', 'school_MS', 'Medu', 'studytime']


X_train_selected = X_train_full[top_features_sp]
X_test_selected = X_test_full[top_features_sp]

print("feature selection aplicada")
print("Novo formato do X_train:", X_train_selected.shape)

feature selection aplicada
Novo formato do X_train: (486, 5)


In [4]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)
print("Dados selecionados e normalizados.")


smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

print("\nNormalização e smote.")
print(pd.Series(y_train_resampled).value_counts())

Dados selecionados e normalizados.

Normalização e smote.
1    411
0    411
Name: count, dtype: int64


In [ ]:



num_features = X_train_resampled.shape[1]  

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
optimizer = COBYLA(maxiter=100)
sampler = StatevectorSampler()


num_parametros = ansatz.num_parameters
rng = np.random.default_rng(42)
initial_point = rng.random(num_parametros)


vqc_sp = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point 
)

print("\nInstância do VQC criada.")


start_time = time.time()
vqc_sp.fit(X_train_resampled, y_train_resampled)
end_time = time.time()
print(f"Modelo VQC treinado em {end_time - start_time:.2f} segundos.")

vqc_predictions_sp = vqc_sp.predict(X_test_scaled)

print("\n Relatório de Classificação para a QNN/VQC (Student Performance(Port))")
print(classification_report(y_test, vqc_predictions_sp, target_names=['Reprovado', 'Aprovado']))

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



Instância do VQC criada. Iniciando treinamento...
Modelo VQC treinado em 522.46 segundos.

--- Relatório de Classificação para a QNN/VQC (Student Performance) ---
              precision    recall  f1-score   support

   Reprovado       0.23      0.52      0.32        25
    Aprovado       0.89      0.68      0.77       138

    accuracy                           0.66       163
   macro avg       0.56      0.60      0.54       163
weighted avg       0.79      0.66      0.70       163

